<a href="https://colab.research.google.com/github/MaiSerry/DoctBot_grad/blob/main/main_code_DoctorBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.1 MB/s

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=72c7ad616f9ae414456d4101af34ee81ac6e5c402ea09b45236c5ce5507f8456
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification,AutoModelForSeq2SeqLM,pipeline
import torch
import whisper
import tensorflow as tf
from langdetect import detect, LangDetectException
import os

In [ ]:
# Global variables for STICKY session output language
SESSION_OUTPUT_LANGUAGE = "en"  # Default to English
SESSION_LANGUAGE_LOCKED = False # To ensure SESSION_OUTPUT_LANGUAGE is set only once


In [ ]:
model_name = "Zabihin/Symptom_to_Diagnosis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
speech_model = whisper.load_model("medium")

#translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ar-en")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/434M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at Zabihin/Symptom_to_Diagnosis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
100%|██████████████████████████████████████| 1.42G/1.42G [00:12<00:00, 120MiB/s]


In [ ]:
disease_to_department = {
    "Influenza": "General Medicine",
    "Pneumonia": "Pulmonology",
    "Anemia": "Hematology",
    "Diabetes Mellitus": "Endocrinology",
    "Hypertension": "Cardiology",
    "Coronary Artery Disease": "Cardiology",
    "Stroke": "Neurology",
    "Migraine": "Neurology",
    "Chronic Kidney Disease": "Nephrology",
    "Liver Disease": "Hepatology / Gastroenterology",
    "Gastroesophageal Reflux": "Gastroenterology",
    "Peptic Ulcer Disease": "Gastroenterology",
    "Irritable Bowel Syndrome": "Gastroenterology",
    "Osteoarthritis": "Rheumatology / Orthopedics",
    "Rheumatoid Arthritis": "Rheumatology",
    "Chronic Obstructive Pulmonary Disease (COPD)": "Pulmonology",
    "Tuberculosis": "Pulmonology / Infectious Diseases",
    "Urinary Tract Infection": "Urology",
    "Sexually Transmitted Infection (STI)": "Urology / Infectious Diseases",
    "Depression": "Psychiatry",
    "Anxiety Disorder": "Psychiatry",
    "Skull Fracture": "Neurology / Orthopedics"
}

In [ ]:
department_translations = {
    "General Medicine": "الطب العام",
    "Pulmonology": "أمراض الصدر",
    "Hematology": "أمراض الدم",
    "Endocrinology": "الطب الباطني / الغدد الصماء",
    "Cardiology": "أمراض القلب",
    "Neurology": "أمراض الأعصاب",
    "Nephrology": "أمراض الكلى",
    "Hepatology / Gastroenterology": "أمراض الكبد / الجهاز الهضمي",
    "Gastroenterology": "الجهاز الهضمي",
    "Rheumatology / Orthopedics": "الروماتيزم / العظام",
    "Rheumatology": "الروماتيزم",
    "Orthopedics": "العظام",
    "Infectious Diseases": "الأمراض المعدية",
    "Urology": "المسالك البولية",
    "Psychiatry": "الطب النفسي",
    "General Practitioner": "طبيب عام"
}


In [ ]:
translation_model_name = "facebook/m2m100_418M"
translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
def translate_text(text, target_lang, source_lang=None):
    global translation_tokenizer, translation_model
    if not text or not text.strip():
        return ""
    if source_lang:
        translation_tokenizer.src_lang = source_lang
    forced_bos_id = translation_tokenizer.get_lang_id(target_lang)
    inputs = translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    translated_tokens = translation_model.generate(**inputs, forced_bos_token_id=forced_bos_id)
    translated_text = translation_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0] #
    return translated_text

In [ ]:
CONFIDENCE_THRESHOLD = 0.60

def predict_disease(symptoms_in_english, top_n=3):
    inputs = tokenizer(symptoms_in_english, return_tensors="tf", truncation=True, padding=True)
    outputs = model(**inputs)
    probabilities = tf.nn.softmax(outputs.logits, axis=1)[0]
    top_n_values_tf, top_n_indices_tf = tf.math.top_k(probabilities, k=top_n)
    top_n_values = top_n_values_tf.numpy()
    top_n_indices = top_n_indices_tf.numpy()
    english_diseases = list(disease_to_department.keys())
    predictions = []

    if not top_n_values.size > 0 or top_n_values[0] < CONFIDENCE_THRESHOLD:
        return [("Insufficient Symptoms", 0.0, "General Practitioner")]

    for i in range(min(top_n, len(top_n_values))):
        disease_index = top_n_indices[i]
        confidence = top_n_values[i]
        predicted_disease_en = english_diseases[disease_index] if 0 <= disease_index < len(english_diseases) else "Unknown"
        department_en = disease_to_department.get(predicted_disease_en, "General Practitioner")
        predictions.append((predicted_disease_en, confidence, department_en))
    return predictions


In [ ]:
def get_symptoms_from_audio(file_path):
    print(get_ui_text("transcribing_audio"))
    if not os.path.exists(file_path):
        print(get_ui_text("audio_file_not_found", file_path=file_path))
        return "", "en" # Return empty symptom and default lang
    try:
        result = speech_model.transcribe(file_path)
        transcribed_text = result["text"].strip()
        detected_lang_of_speech = result.get("language", "en")
        print(get_ui_text("transcribed_text_label", lang=detected_lang_of_speech, text=transcribed_text))

        if not transcribed_text:
            print(get_ui_text("transcription_empty"))
            return "", detected_lang_of_speech

        symptoms_for_model = transcribed_text
        if detected_lang_of_speech == "ar":
            symptoms_for_model = translate_text(transcribed_text, "en", "ar")
            print(get_ui_text("translated_to_en_label", text=symptoms_for_model))

        return symptoms_for_model, detected_lang_of_speech
    except Exception as e:
        print(get_ui_text("transcription_error_detailed", e=str(e)))
        return "", "en"


In [ ]:
# -- Text translations --

ui_texts = {
    "en": {
        "welcome": "👋 Welcome to AI Medical Diagnosis System!",
        "input_method_prompt": "Would you like to enter symptoms as text or speech? (text/speech): ",
        "invalid_input_method": "Invalid input. Please type \"text\" or \"speech\".",
        "speech_file_prompt": "🎙 Please provide the path to your recorded audio file: ",
        "audio_transcription_error": "Could not transcribe audio, audio was empty, or an error occurred. Please try again or switch to text.",
        "no_audio_file": "No audio file path provided.",
        "text_symptoms_prompt": "🩺 Please describe your symptoms: ",
        "symptoms_required": "\nSymptoms are required to proceed. Please provide your symptoms.",
        "must_provide_symptoms": "You must provide symptoms to continue.",
        "insufficient_symptoms_msg": "\n⚠️ Your symptoms are not specific enough for a diagnosis.",
        "recommended_dept_insufficient": "🏥 Recommended Department: General Practitioner",
        "provide_more_details": "💬 Please provide more details (e.g., duration, pain location, severity).",
        "add_more_symptoms_prompt": "➕ Add more symptoms (or press Enter to stop): ",
        "diagnosis_results_header": "\n✅ Diagnosis Results:",
        "disease_label": "🩺 Disease: ",
        "confidence_label": " (confidence:{confidence:.2f})",
        "recommended_dept_label": "   🏥 Recommended Department: ",
        "audio_file_not_found": "Audio file not found at: {file_path}",
        "transcription_empty": "Transcription resulted in empty text.",
        "transcription_error_detailed": "Error during audio transcription or translation: {e}",
        "transcribing_audio": "🎤 Transcribing audio...",
        "transcribed_text_label": "🗣 Transcribed Text ({lang}): {text}",
        "translated_to_en_label": "🌍 Translated to English for model: {text}",
        "detected_text_lang_label": "Detected input language (text): {lang}",
        "lang_detection_error": "Could not detect language from text, assuming English for model. UI will use default.",
        "session_lang_set_debug": "Session language locked to: {lang}",
        "no_further_symptoms": "No further symptoms provided. Exiting diagnosis."

    },
    "ar": {
        "welcome": "👋 مرحبًا بك في نظام التشخيص الطبي بالذكاء الاصطناعي!",
        "input_method_prompt": "هل ترغب في إدخال الأعراض كنص أم كلام؟ (text/speech): ",
        "invalid_input_method": "إدخال غير صالح. يرجى كتابة \"text\" أو \"speech\".",
        "speech_file_prompt": "🎙 يرجى تقديم مسار ملفك الصوتي المسجل: ",
        "audio_transcription_error": "تعذر نسخ الصوت، أو كان الصوت فارغًا، أو حدث خطأ. يرجى المحاولة مرة أخرى أو التبديل إلى النص.",
        "no_audio_file": "لم يتم تقديم مسار ملف صوتي.",
        "text_symptoms_prompt": "🩺 يرجى وصف الأعراض الخاصة بك: ",
        "symptoms_required": "\nالأعراض مطلوبة للمتابعة. يرجى تقديم الأعراض الخاصة بك.",
        "must_provide_symptoms": "يجب عليك تقديم الأعراض للمتابعة.",
        "insufficient_symptoms_msg": "\n⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.",
        "recommended_dept_insufficient": "🏥 القسم الموصى به: طبيب عام",
        "provide_more_details": "💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).",
        "add_more_symptoms_prompt": "➕ أضف المزيد من الأعراض (أو اضغط على Enter للتوقف): ",
        "diagnosis_results_header": "\n✅ نتائج التشخيص:",
        "disease_label": "🩺 المرض: ",
        "confidence_label": " (الثقة: {confidence:.2f})",
        "recommended_dept_label": "   🏥 القسم الموصى به: ",
        "audio_file_not_found": "لم يتم العثور على الملف الصوتي في: {file_path}",
        "transcription_empty": "أسفر النسخ عن نص فارغ.",
        "transcription_error_detailed": "خطأ أثناء نسخ الصوت أو الترجمة: {e}",
        "transcribing_audio": "🎤 جاري نسخ الصوت...",
        "transcribed_text_label": "🗣 النص المنسوخ ({lang}): {text}",
        "translated_to_en_label": "🌍 تمت الترجمة إلى الإنجليزية للنموذج: {text}",
        "detected_text_lang_label": "تم اكتشاف لغة الإدخال (النص): {lang}",
        "lang_detection_error": "لم نتمكن من تحديد اللغة من النص، سنفترض الإنجليزية للنموذج. ستستخدم الواجهة اللغة الافتراضية.",
        "session_lang_set_debug": "تم تثبيت لغة الجلسة على: {lang}",
        "no_further_symptoms": "لم يتم تقديم المزيد من الأعراض. إنهاء التشخيص."
    }
}

In [ ]:
def get_ui_text(key, **kwargs):
    global SESSION_OUTPUT_LANGUAGE
    #return ui_texts.get(SESSION_OUTPUT_LANGUAGE, ui_texts["en"]).get(key, key).format(**kwargs)

    template = ui_texts.get(SESSION_OUTPUT_LANGUAGE, ui_texts["en"]).get(key, key)
    try:
      return template.format(**kwargs)
    except (IndexError, KeyError):
      return template

In [ ]:

def set_session_language_if_not_locked(detected_language_code):
    global SESSION_OUTPUT_LANGUAGE, SESSION_LANGUAGE_LOCKED
    if not SESSION_LANGUAGE_LOCKED:
        if detected_language_code == "ar":
            SESSION_OUTPUT_LANGUAGE = "ar"
        else:
            SESSION_OUTPUT_LANGUAGE = "en"
        SESSION_LANGUAGE_LOCKED = True
        # print(get_ui_text("session_lang_set_debug", lang=SESSION_OUTPUT_LANGUAGE)) # For debugging

In [ ]:
def interactive_diagnosis():
    global SESSION_OUTPUT_LANGUAGE, SESSION_LANGUAGE_LOCKED
    SESSION_LANGUAGE_LOCKED = False
    SESSION_OUTPUT_LANGUAGE = "en"

    print(get_ui_text("welcome"))

    input_method = ""
    while input_method not in ["text", "speech"]:
        input_method = input(get_ui_text("input_method_prompt")).strip().lower()
        if input_method not in ["text", "speech"]:
            print(get_ui_text("invalid_input_method"))

    accumulated_symptoms_for_model = ""
    initial_symptoms_obtained = False

    while not initial_symptoms_obtained:
        current_symptoms_for_model_segment = ""
        detected_lang_for_segment = "en"

        if input_method == "speech":
            file_path = input(get_ui_text("speech_file_prompt")).strip()
            if file_path:
                current_symptoms_for_model_segment, detected_lang_for_segment = get_symptoms_from_audio(file_path)
                if not current_symptoms_for_model_segment:
                    print(get_ui_text("audio_transcription_error"))
            else:
                print(get_ui_text("no_audio_file"))
        elif input_method == "text":
            raw_symptoms = input(get_ui_text("text_symptoms_prompt")).strip()
            if raw_symptoms:
                try:
                    detected_lang_for_segment = detect(raw_symptoms)
                    print(get_ui_text("detected_text_lang_label", lang=detected_lang_for_segment))
                    if detected_lang_for_segment == "ar":
                        current_symptoms_for_model_segment = translate_text(raw_symptoms, "en", "ar")
                        print(get_ui_text("translated_to_en_label", text=current_symptoms_for_model_segment))
                    else:
                        current_symptoms_for_model_segment = raw_symptoms
                except LangDetectException:
                    print(get_ui_text("lang_detection_error"))
                    detected_lang_for_segment = "en" # Default to English for UI if detection fails
                    current_symptoms_for_model_segment = raw_symptoms
            else:
                 print(get_ui_text("symptoms_required")) # if no text symptoms entered

        if current_symptoms_for_model_segment:
            set_session_language_if_not_locked(detected_lang_for_segment)
            accumulated_symptoms_for_model = current_symptoms_for_model_segment
            initial_symptoms_obtained = True
        else:
            print(get_ui_text("must_provide_symptoms"))

    #diagnosis loop - accumulates symptoms
    while True:
        if not accumulated_symptoms_for_model.strip():
            # This should ideally not be reached if initial_symptoms_obtained is true
            # but as a fallback, or if user clears symptoms somehow.
            print(get_ui_text("must_provide_symptoms"))
            # Re-prompt for initial symptoms if empty, respecting original input_method choice
            # This part could be a call to a function that gets initial symptoms again.
            # For now, let's assume the loop for initial symptoms ensures it's not empty.
            # If it is, we might need to break or re-enter the initial symptom gathering phase.
            # For simplicity, if it's empty here, we break, as the user might have pressed Enter to stop earlier.
            break

        predictions_en = predict_disease(accumulated_symptoms_for_model)

        if predictions_en[0][0] == "Insufficient Symptoms":
            print(get_ui_text("insufficient_symptoms_msg"))
            print(get_ui_text("recommended_dept_insufficient"))
            print(get_ui_text("provide_more_details"))
            more_info_raw = input(get_ui_text("add_more_symptoms_prompt")).strip()

            if not more_info_raw:
                print(get_ui_text("no_further_symptoms"))
                break

            more_symptoms_en = more_info_raw
            try:
                # Language of additional symptoms does not change SESSION_OUTPUT_LANGUAGE
                lang_of_more_info = detect(more_info_raw)
                if lang_of_more_info == "ar":
                    more_symptoms_en = translate_text(more_info_raw, "en", "ar")
                    # Optional: print translation confirmation for additional symptoms
                    # print(f"(Additional symptoms translated to EN: {more_symptoms_en})")
            except LangDetectException:
                 pass # Assume English or non-translatable, add as is for model

            accumulated_symptoms_for_model += " " + more_symptoms_en.strip()
            # print(f"(Updated symptoms for model: {accumulated_symptoms_for_model})") # For debugging
        else:
            print(get_ui_text("diagnosis_results_header"))
            for i, (disease_en, confidence, department_en) in enumerate(predictions_en):
                disease_display = disease_en
                department_display = department_en
                if SESSION_OUTPUT_LANGUAGE == "ar":
                    disease_display = translate_text(disease_en, "ar", "en")
                    department_display = department_translations.get(department_en, department_en)

                # Corrected f-string usage with single quotes for get_ui_text calls
                print(f"{i+1}. {get_ui_text('disease_label')}{disease_display}{get_ui_text('confidence_label', confidence=confidence)}")
                print(f"   {get_ui_text('recommended_dept_label')}{department_display}\n")

            break


In [ ]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text 
🩺 Please describe your symptoms: صداع وضيق في التنفس و غثيان و قيء
Detected input language (text): ar
🌍 Translated to English for model: Headache, difficulty in breathing, nausea and vomiting.

⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.
🏥 القسم الموصى به: طبيب عام
💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).
➕ أضف المزيد من الأعراض (أو اضغط على Enter للتوقف): تعب وإرهاق

✅ نتائج التشخيص:
1. 🩺 المرض: الأنفلونزا (الثقة: 0.61)
      🏥 القسم الموصى به: الطب العام

2. 🩺 المرض: مرض السكري Mellitus (الثقة: 0.08)
      🏥 القسم الموصى به: الطب الباطني / الغدد الصماء

3. 🩺 المرض: مرض القرحة الببتية (الثقة: 0.05)
      🏥 القسم الموصى به: الجهاز الهضمي



In [ ]:
if __name__ == "__main__":

    interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: text 
Detected input language (text): ca

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): I have headaches and nausea

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): I have severe headaches, nausea, and dizziness

✅ Diagnosis Results:
1. 🩺 Disease: Peptic Ulcer Disease (confidence:0.71)
      🏥 Recommended Department: Gastroenterology

2. 🩺 Disease: Chronic Obstructive Pulmonary Disease (COPD) (confidence:0.12)
      🏥 Recommended Department: Pulmonology

3. 🩺 Disease: Depression (confidence:0.0

In [ ]:
if __name__ == "__main__":

    interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: I have a sick throat and fever
🔄 Translated Symptoms: I have a sick throat and fever

✅ Diagnosis Results:
1. 🩺 Disease: Influenza (Confidence: 0.91)
   🏥 Recommended Department: General Medicine

2. 🩺 Disease: Coronary Artery Disease (Confidence: 0.02)
   🏥 Recommended Department: Cardiology

3. 🩺 Disease: Gastroesophageal Reflux (Confidence: 0.01)
   🏥 Recommended Department: Gastroenterology



In [ ]:
## old excute from old code

In [ ]:
interactive_diagnosis()


👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): speech
🎙 Please provide the path to your recorded audio file: /content/2025-05-15 at 19.38.15_85a89c4e.waptt.opus
🎤 Transcribing audio...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣 Transcribed Text (ar): وعيني من صدور ولا استطيع أنه يرتكب جسدي واشعر أحيانا بالدور
🌍 Translated to English for model: My eye is out of the chest and I can’t do it my body and sometimes I feel the role

⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.
🏥 القسم الموصى به: طبيب عام
💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).
➕ أضف المزيد من الأعراض (أو اضغط على Enter للتوقف): لا أستطيع النوم , أعاني من صداع

⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.
🏥 القسم الموصى به: طبيب عام
💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).
➕ أضف المزيد من الأعراض (أو اضغط على Enter للتوقف): رتجف جسدي كله

⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.
🏥 القسم الموصى به: طبيب عام
💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).
➕ أضف المزيد من الأعراض (أو اضغط على Enter للتوقف): أشعر أحيانًا بدوار

⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.
🏥 القسم الموصى به: طبيب عام
💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).
➕ أض

In [ ]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: أعاني من صداع وغثيان
🔄 Translated Symptoms: I have headaches and nausea

✅ Diagnosis Results:
1. 🩺 Disease: Chronic Obstructive Pulmonary Disease (COPD) (Confidence: 0.84)
   🏥 Recommended Department: Pulmonology

2. 🩺 Disease: Stroke (Confidence: 0.03)
   🏥 Recommended Department: Neurology

3. 🩺 Disease: Chronic Kidney Disease (Confidence: 0.02)
   🏥 Recommended Department: Nephrology



In [ ]:
## fine-tuning data



# Load model and tokenizer
model_name = "Zabihin/Symptom_to_Diagnosis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)


RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	size mismatch for bert.embeddings.position_embeddings.weight: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([512, 768]).
	size mismatch for bert.embeddings.token_type_embeddings.weight: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([2, 768]).
	size mismatch for bert.embeddings.LayerNorm.weight: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for bert.embeddings.LayerNorm.bias: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for bert.encoder.layer.0.attention.self.query.weight: copying a param with shape torch.Size([28996, 768]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch f